In [362]:
# Импорт библиотек
import numpy as np
from datasets import Dataset
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric
import pandas as pd

In [363]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 111700
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 112
    })
})

In [364]:
# загрузка параметров
raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
nltk.download('punkt')

  0%|          | 0/3 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [425]:
# Ввести свой ключ huggingface hyb
from huggingface_hub import notebook_login
notebook_login()

In [664]:
# Определение параметров
REPO = "speller-t5-9001"  # Введите наазвание название репозитория
MODEL_NAME =  "sberbank-ai/ruT5-base" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 8

In [613]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [614]:
import pandas as pd
# data = pd.read_csv("./dataset_big.csv", index_col=0).drop_duplicates()
data= pd.read_csv("./dataset_small 2.csv", index_col=0).drop_duplicates()
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()

In [615]:

import pandas as pd
# data = pd.read_csv("./dataset_big.csv", index_col=0).drop_duplicates()[:15000]
# data = pd.read_csv("./dataset_small 2.csv", index_col=0).drop_duplicates()
test_data = pd.read_csv("./test_dataset.csv", index_col=0).drop_duplicates()


In [616]:
data=data

In [617]:
print(data.shape)
print(test_data.shape)

(117417, 2)
(2005, 2)


In [618]:
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import AutoTokenizer, AutoModel, T5Tokenizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
# model.resize_token_embeddings(len(tokenizer));
# model = nn.DataParallel(model).to(device)

In [619]:
input_data = list(data.source)
output_data = list(data.target)
task_prefix = "Исправь: "
input_sequences = input_data
encoding = tokenizer( [task_prefix + sequence for sequence in input_sequences], padding="longest", \
                     max_length=MAX_INPUT, truncation=True, return_tensors="pt", )
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
target_encoding = tokenizer(output_data, padding='max_length', max_length=MAX_OUTPUT, truncation=True)
labels = target_encoding.input_ids
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100
data = Dataset.from_pandas(pd.DataFrame({'input_ids': list(np.array(input_ids)), 'attention_mask': list(np.array(attention_mask)), \
                                         'labels': list(np.array(labels))}))
data = data.train_test_split(0.001)

In [620]:
max(list(map(lambda x:len(x.split()),input_data)))

101

In [622]:
REPO

'speller-t5-9001'

In [665]:
model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности
# Закоментить после первого соъранения в репозиторий свой необъязательно
tokenizer.push_to_hub(REPO)

CommitInfo(commit_url='https://huggingface.co/summervent/speller-t5-9001/commit/73a91d406b1795b20fbb509600608d55051557a3', commit_message='Upload tokenizer', commit_description='', oid='73a91d406b1795b20fbb509600608d55051557a3', pr_url=None, pr_revision=None, pr_num=None)

In [624]:
train = data['train']
test = data['test'] # Уменьшил так тестовыу. выборку чтоб не ждать долго расчет ошибок между эпохами

In [625]:
test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 118
})

In [626]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #return_tensors="tf"

In [627]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  
  # Rouge expects a newline after each sentence
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
  
  result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  # Extract a few results
  result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  
  # Add mean generated length
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)
  
  return {k: round(v, 4) for k, v in result.items()}

In [628]:
REPO

'speller-t5-9001'

In [629]:
training_args = Seq2SeqTrainingArguments(
  output_dir = REPO,
  #overwrite_output_dir=True,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  logging_steps=10,
  eval_steps=500,
  save_steps=5000,
  num_train_epochs=1,
  predict_with_generate=True,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  fp16=True,
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  # logging_strategy='steps',
  #logging_dir='logs',
  push_to_hub=True,
  report_to='tensorboard'
)

In [634]:
# Выберем вручную оптимизатор. Т5 в оригинальной архитектуре использует Адафактор оптимизатор
optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

In [635]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = data['train'],
  eval_dataset = data['test'],
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics,
)

In [636]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING)
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,1.172000,0.565939,14.166900,4.126500,13.787800,14.104400,42.745800
1000,0.706300,0.420709,14.563800,4.830500,14.468800,14.590700,43.889800
1500,0.660400,0.355705,16.267200,4.868500,16.230800,16.351600,43.864400
2000,0.542900,0.326649,16.643600,5.116100,16.666700,16.687200,43.483100
2500,0.524500,0.296367,16.666700,5.196300,16.677500,16.770700,42.398300
3000,0.581200,0.275728,16.696900,5.339000,16.733100,16.844900,41.305100
3500,0.501900,0.262573,16.686000,5.446200,16.681500,16.873300,40.720300
4000,0.418200,0.253105,16.714200,5.508500,16.666700,16.937300,40.610200
4500,0.459200,0.241299,16.947000,5.540400,16.958100,17.059000,40.144100
5000,0.462600,0.229852,16.949200,5.606300,16.944000,17.023500,40.347500


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=14663, training_loss=0.5018231812159524, metrics={'train_runtime': 5945.9758, 'train_samples_per_second': 19.727, 'train_steps_per_second': 2.466, 'total_flos': 3.571507580239872e+16, 'train_loss': 0.5018231812159524, 'epoch': 1.0})

In [393]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING)
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,1.022700,0.541063,17.620100,7.118600,17.655400,17.584700,45.542400
1000,0.722400,0.426860,18.149700,7.118600,18.149700,17.973200,42.779700
1500,0.710100,0.354232,18.997200,7.966100,18.997200,18.961900,42.398300
2000,0.596200,0.328305,18.997200,7.966100,18.997200,18.961900,42.254200
2500,0.535000,0.310411,18.997200,7.966100,18.997200,18.961900,42.262700
3000,0.612400,0.284283,18.997200,7.966100,18.997200,18.961900,42.491500
3500,0.491000,0.270633,18.997200,7.966100,18.997200,18.961900,42.432200
4000,0.502800,0.264690,19.542900,8.587600,19.542900,19.562100,42.389800
4500,0.454700,0.254836,18.997200,7.966100,18.997200,18.961900,42.178000
5000,0.433500,0.244784,19.542900,8.587600,19.542900,19.562100,42.178000


/root/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


TrainOutput(global_step=14663, training_loss=0.5125837166093263, metrics={'train_runtime': 5287.5612, 'train_samples_per_second': 22.184, 'train_steps_per_second': 2.773, 'total_flos': 3.571507580239872e+16, 'train_loss': 0.5125837166093263, 'epoch': 1.0})

In [410]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# or 
logging.disable(logging.WARNING)
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.273900,0.168878,19.632800,8.757100,19.597500,19.632800,41.313600
1000,0.322300,0.166947,19.350300,8.333300,19.350300,19.350300,41.203400
1500,0.344200,0.168186,19.632800,8.757100,19.597500,19.632800,41.271200
2000,0.322100,0.164737,19.350300,8.333300,19.350300,19.350300,41.220300
2500,0.304800,0.163489,19.632800,8.757100,19.597500,19.632800,41.313600
3000,0.257500,0.161220,19.632800,8.757100,19.597500,19.632800,41.262700
3500,0.293400,0.161415,19.632800,8.757100,19.597500,19.632800,41.271200
4000,0.280600,0.159967,19.632800,8.757100,19.597500,19.632800,41.279700
4500,0.252500,0.159849,19.632800,8.757100,19.597500,19.632800,41.381400
5000,0.292900,0.156058,19.632800,8.757100,19.597500,19.632800,41.228800


TrainOutput(global_step=14663, training_loss=0.2609096582106774, metrics={'train_runtime': 5242.6271, 'train_samples_per_second': 22.374, 'train_steps_per_second': 2.797, 'total_flos': 3.571507580239872e+16, 'train_loss': 0.2609096582106774, 'epoch': 1.0})

In [421]:
REPO

'speller-t5-900'

In [637]:
trainer.push_to_hub(REPO)

Upload file pytorch_model.bin:   0%|          | 32.0k/850M [00:00<?, ?B/s]

Upload file runs/Feb03_22-53-30_anaslan/events.out.tfevents.1675465106.anaslan.23680.45:  13%|#3        | 32.0…

'https://huggingface.co/summervent/speller-t5-9001/commit/d451efb8a4c569bfd12079e2edd6a408bb252b9b'

In [666]:
from transformers import T5TokenizerFast, AutoModelForSeq2SeqLM
MODEL_NAME_ =  "summervent/speller-t5-9001"
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME_)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_).to(device)

# model.resize_token_embeddings(len(tokenizer));


In [667]:
MAX_INPUT = 256
input_sequences = list(test_data.source.values)

task_prefix = "Исправь: "
encoded = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=MAX_INPUT,
    truncation=True,
    return_tensors="pt",
)
test_loader = torch.utils.data.DataLoader(input_sequences, batch_size=8)

predicts__ = []
for input_sequences in tqdm(test_loader):
    encoded = tokenizer(
        [task_prefix + sequence for sequence in input_sequences],
        padding="longest",
        max_length=MAX_INPUT,
        truncation=True,
        return_tensors="pt",
    )

    predicts_ = model.generate(**encoded.to(device), max_length=len(encoded['input_ids'][0]), repetition_penalty=1.0)
    
    predicts_ = tokenizer.batch_decode(predicts_, skip_special_tokens=True)
    predicts__ += predicts_


100%|██████████| 251/251 [03:54<00:00,  1.07it/s]


In [668]:
idx = 82
print(test_data.source[idx])
print(test_data.target[idx])
print()
print(predicts__[idx])

хотелось бы более подробно узнать о алексее шинкоренко опыт работы в области фотографии опыт работы в преподавателем желатьльно со ссылкой на портфолио
хотелось бы более подробно узнать о алексее шинкоренко опыт работы в области фотографии опыт работы преподавателем желательно со ссылкой на портфолио

хотелось бы более подробно узнать о Алексее шинкоренко опыт работы в области фотографии опыт работы в преподавателем желательно со ссылкой на портфолио


In [669]:
train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 117299
})

In [670]:
from evaluate_spell import extract_words, make_corrections_data, measure_quality


OUTPUT_DIFF = False


source_sents = [
    extract_words(line.strip().strip("\ufeff"))
    for line in test_data.source
    if line.strip().strip("\ufeff") != ""
]
correct_sents = [
    extract_words(line.strip().strip("\ufeff"))
    for line in test_data.target
    if line.strip().strip("\ufeff") != ""
]
answer_sents = [extract_words(line.strip().strip("\ufeff")) for line in predicts__]
etalon_corrections, answer_corrections = make_corrections_data(
    source_sents, correct_sents, answer_sents
)
TP, precision, recall, f_measure = measure_quality(
    etalon_corrections, answer_corrections
)
print(
    f"Precision={100*precision:.2f} Recall={100 * recall:.2f} FMeasure={100 * f_measure:.2f}"
)
print(TP, len(answer_corrections), len(etalon_corrections))
if OUTPUT_DIFF:
    output_differences(
        "diff.txt",
        source_sents,
        correct_sents,
        answer_sents,
        etalon_corrections,
        answer_corrections,
    );

0 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 
181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 
261 262 263 264 265 266 267 268 269 270 271 272 273 

In [671]:
count_correct = 0
for answer, correct in zip(correct_sents, answer_sents):
    if answer == correct:
        count_correct += 1
print(count_correct / 2005)

0.548
